# Creating cluster member lists

In [47]:
import numpy as np

# path to cluster catalog
cluster_cat_path = "camira_s19a_wide_sm_v1.txt"
# path to cluster members catalog
cluster_mem_path = "camira_mem_s19a_wide_sm_v1.dat"

In [60]:
HSC_cl_cat = np.loadtxt(cluster_cat_path, usecols=(0,1,2,3,4,5)) 
HSC_cl_cat_IDs = np.loadtxt(cluster_cat_path, dtype='str', usecols=(6)) 

cluster_mems = np.loadtxt(cluster_mem_path)

## Increasing efficiency

Idea: I could create a dictionary that pairs a cluster RA with the pair (a, b), where a is the RA's lowest index in the list and b is the highest

In [49]:
# cluster_mem_path_test ='members_cat_test.txt'
# cluster_mems = np.loadtxt(cluster_mem_path_test) # test file 

## Dictionary of cluster props to cluster ID

In [73]:
list_props_ID = []

for i in range(len(HSC_cl_cat)):
    RA = HSC_cl_cat[i, 0]
    dec = HSC_cl_cat[i, 1]
    # z in the full member list is rounded to 3 decimal places
    z = np.round(HSC_cl_cat[i, 2], 3)
    ID = HSC_cl_cat_IDs[i]
    props = (RA, dec, z)
    
    list_props_ID.append((props, ID))
    
dict_props_ID = dict(list_props_ID)

In [77]:
# this dictionary stores the key-value pair of (cluster ID)-slice(a, b), where: 
# 1. cluster ID is found via dict_props_ID
# 2. a is the first place in the full member list where the cluster RA appears 
# and b-1 is the last
list_ID_slice = []
low_ind = 0 # a
high_ind = 0 # b

num_of_cls = 0

for i in range(len(cluster_mems)):
    cluster_RA = cluster_mems[i, 0]
    cluster_dec = cluster_mems[i, 1]
    cluster_z = cluster_mems[i, 3]
    
    if (i < (len(cluster_mems) - 1)): 
        if (cluster_RA == cluster_mems[i+1, 0]): continue

    num_of_cls += 1
    # if we've reached this point, it means we've reached the last galaxy 
    # in this cluster
    
    props = (cluster_RA, cluster_dec, cluster_z)
    ID = dict_props_ID[props]
    
    high_ind = i + 1
    list_ID_slice.append((ID, slice(low_ind, high_ind)))
    low_ind = high_ind
    
dict_ID_slice = dict(list_ID_slice)

KeyError: (31.542414, -6.095341, 1.159)

In [81]:
num_of_cls

59

In [82]:
# set_of_RAs = set(cluster_mems[:,0])
# for i in range(len(HSC_cl_cat)):
# #     cl_RA = HSC_cl_cat[i, 0]
# #     if cl_RA not in set_of_RAs:
# #         print(cl_RA)
#     if (i == len(HSC_cl_cat) - 1): continue
#     if HSC_cl_cat[i, 0] == HSC_cl_cat[i+1, 0]:
#         print(f'Index: {i}')
#         print(f'Repeated RA: {HSC_cl_cat[i, 0]}')

In [39]:
set_of_RAs2 = set(HSC_cl_cat[:, 0])
len(set_of_RAs2)

16581

In [14]:
# main method
def get_members(ra, dec, z, prob_cutoff=0, print_info=False):
    '''
    Given an HSC cluster's RA, declination, and z, save a list containing
    all of its members.
    
    ARGUMENTS
    ----------
    ra: right ascension (degrees) of cluster
    
    dec: declination (degrees) of cluster
    
    prob_cutoff: exclude members with a probability membership smaller 
    than prob_cutoff; default is 0 (i.e. no members excluded)
    
    print_info: print information about the cluster and its member 
    list; default is False. Useful for debugging.
    
    '''
    
    # TODO: load the members catalog into a numpy array
    cluster_mems = np.readtxt(cluster_mem_path)
    
    # TODO: identify members who match the given cluster RA/dec
    # (do I get the indices in cluster_mem_path?)
    
    # TODO: omit clusters whose membership probabilities are too low
    # membership_probs = cluster_mems[:,7]
    
    
    # TODO: create the members list
    # (do I write to a new file? or can I slice into the full member list?)

    
    # TODO: save members as a new file
    member_list_path = "Member_Lists/memlist_ra=" + str(np.round(ra, 3)) + \
        "_dec=" + str(np.round(dec, 3))
    #...
    
    if print_info:
        print('Cluster members saved.')